# Train a gesture recognizer

We start by installing pre-reqs. We use a TypeScript program to segment data (can be replaced in future).

In [12]:
! npm install -g -u typescript ts-node ml4f
#! pip3 install tensorflow==2.3 tensorboard=2.3
#! npm install @types/node

npm WARN deprecated core-js@1.2.7: core-js@<3 is no longer maintained and not recommended for usage due to the number of issues. Please, upgrade your dependencies to the actual version of core-js@3.
/usr/local/bin/ml4f -> /usr/local/lib/node_modules/ml4f/ml4ffinalize /usr/local/lib/node_modulesKK
/usr/local/bin/ts-node -> /usr/local/lib/node_modules/ts-node/dist/bin.js
/usr/local/bin/ts-script -> /usr/local/lib/node_modules/ts-node/dist/bin-script-deprecated.js
/usr/local/bin/ts-node-transpile-only -> /usr/local/lib/node_modules/ts-node/dist/bin-transpile.js
/usr/local/bin/ts-node-script -> /usr/local/lib/node_modules/ts-node/dist/bin-script.js
/usr/local/bin/tsc -> /usr/local/lib/node_modules/typescript/bin/tsc
/usr/local/bin/tsserver -> /usr/local/lib/node_modules/typescript/bin/tsserver
+ ml4f@1.4.0
+ ts-node@9.0.0
+ typescript@4.0.3
updated 3 packages in 6.501s


Then we run the segmentation script.

In [13]:
! rm -rf built/
! ts-node segment.ts

loading data/michal/punch.csv
steady: [ 0.99609375, 0.04296875, 0.04296875 ]
cutoff: 0.453125 in cutoff %: 55.947136563876654
loading data/michal/right.csv
steady: [ 0.9375, 0.33203125, 0.0234375 ]
cutoff: 0.5703125 in cutoff %: 61.8100447538538
loading data/michal/left.csv
steady: [ 0.98828125, 0.01171875, 0.08984375 ]
cutoff: 0.4609375 in cutoff %: 63.18245450303313
loading data/michal/noise.csv
loading data/michal/noise1.csv
loading data/ira/punch2.csv
steady: [ 1.00390625, -0.08984375, 0.00390625 ]
cutoff: 1.2109375 in cutoff %: 67.358934169279
loading data/ira/right1.csv
steady: [ 0.97265625, 0.19140625, 0.21484375 ]
cutoff: 0.546875 in cutoff %: 63.898557931377425
loading data/ira/left0.csv
steady: [ 0.9765625, -0.15625, 0.22265625 ]
cutoff: 0.5625 in cutoff %: 58.14591332967636
loading data/ira/noise0.csv
median len: 30
len 50+: 1
data/michal/left.csv drop 1 too long
punch 11 13 10
right 16 19 15
left 15 17 14
noise 7 8 7
noise 19 22 18
punch 19 22 18
right 14 16 13
left 15 18 1

Here, we display the results of segmentation.

In [29]:
import pandas as pd
import matplotlib
import glob

for fn in glob.glob("./built/seg/*.csv"):
    data = pd.read_csv(fn)
    data.plot(figsize=(20,7),title=fn)


## Configure Defaults

In [3]:
# Define paths to model files
import os
# Set a "seed" value, so we get the same random numbers each time we run this
# notebook for reproducible results.
# Numpy is a math library
import numpy as np
np.random.seed(1) # numpy seed
# TensorFlow is an open source machine learning library
import tensorflow as tf
tf.random.set_seed(1) # tensorflow global random seed
# Keras is TensorFlow's high-level API for deep learning
from tensorflow import keras
# Matplotlib is a graphing library
import matplotlib.pyplot as plt
# Math is Python's math library
import math

Set Seed for Repeatable Results
## Load data to numpy

In [4]:
class_names = ['noise', 'punch', 'left', 'right']

def load_data(folder):
    x_tmp = []
    y_tmp = []
    for fn in glob.glob(folder + "/*.csv"):
        csv = pd.read_csv(fn)
        assert csv.values.shape == (50,3)
        x_tmp.append(csv.values.astype(np.float32))
        yy = [1 if class_names[i] in fn else 0 for i in range(len(class_names))]
        assert sum(yy) == 1
        y_tmp.append(yy)
    print("load %d samples from %s" % (len(y_tmp), folder))
    return np.array(x_tmp).reshape((-1,50,3,1)), np.array(y_tmp).astype(np.float32)

xs_train, ys_train = load_data("built/train")
xs_test, ys_test = load_data("built/test")
xs_validate, ys_validate = load_data("built/validate")

print(xs_train.shape, ys_train.shape)


load 1000 samples from built/train
load 500 samples from built/test
load 500 samples from built/validate
(1000, 50, 3, 1) (1000, 4)


## Build the model

In [5]:
model = tf.keras.Sequential()

model.add(keras.layers.Conv2D(16, (4, 3),
    input_shape = (50, 3, 1),
    strides= 1,
    activation= 'relu'))

model.add(keras.layers.MaxPooling2D(pool_size = (2, 1)))
model.add(keras.layers.Dropout(0.1))

model.add(keras.layers.Conv2D(16, (2, 1),
    strides= 1,
    activation= 'relu'))

#model.add(keras.layers.MaxPooling2D(pool_size = (2, 1))
model.add(keras.layers.Dropout(0.1))

model.add(keras.layers.Conv2D(16, (2, 1),
    strides= 1,
    activation= 'relu'))
model.add(keras.layers.Dropout(0.1))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(len(class_names), 
    activation= 'softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

w0 = model.get_weights()


## Train the Model




In [6]:

! rm -rf logs/
%load_ext tensorboard
%tensorboard --logdir logs/fit 
#--host localhost --port 8888
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.set_weights(w0)
history = model.fit(xs_train, ys_train, epochs=200, batch_size=32,
                    validation_data=(xs_validate, ys_validate),
                    callbacks=[tensorboard_callback])

Reusing TensorBoard on port 6007 (pid 17933), started 5:06:41 ago. (Use '!kill 17933' to kill it.)

Epoch 1/200
 1/32 [..............................] - ETA: 0s - loss: 1.4062 - accuracy: 0.2188WARNING:tensorflow:From /usr/local/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
32/32 [==============================] - 0s 8ms/step - loss: 1.2629 - accuracy: 0.4510 - val_loss: 1.1565 - val_accuracy: 0.5140
Epoch 2/200
32/32 [==============================] - 0s 3ms/step - loss: 1.1306 - accuracy: 0.5060 - val_loss: 1.0688 - val_accuracy: 0.5220
Epoch 3/200
32/32 [==============================] - 0s 3ms/step - loss: 1.0130 - accuracy: 0.5500 - val_loss: 0.9456 - val_accuracy: 0.5840
Epoch 4/200
32/32 [==============================] - 0s 3ms/step - loss: 0.8923 - accuracy: 0.5860 - val_loss: 0.8307 - val_accuracy: 0.6260
Epoch 5/200
32/32 [==============================] - 0s 2ms/step - loss: 0.7

In [23]:
import json
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

with open('built/test.json', 'w') as outfile:
    json.dump({"x": xs_test, "y": ys_test}, outfile, cls=NumpyEncoder)


In [26]:
model.save("built/model.h5")

# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
model_no_quant_tflite = converter.convert()

# # Save the model to disk
open("built/model-float32.tflite", "wb").write(model_no_quant_tflite)

# Convert the model to the TensorFlow Lite format with quantization
def representative_dataset():
  for i in range(500):
    sample = xs_train[i].reshape(1,50,3,1)
    yield([sample])
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization (except inputs/outputs which are always float)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()

# Save the model to disk
open("built/model-int8.tflite", "wb").write(model_tflite)

! ml4f           --basename model-float32 --eval built/test.json built/model.h5
# to speed things up, use the tfjs model from previous step
! ml4f --float16 --basename model-float16 --eval built/test.json built/converted.tfjs


INFO:tensorflow:Assets written to: /var/folders/43/4pdy4jr56rx76bghv9vbsx140000gn/T/tmpy78vg8pq/assets


INFO:tensorflow:Assets written to: /var/folders/43/4pdy4jr56rx76bghv9vbsx140000gn/T/tmpy78vg8pq/assets


INFO:tensorflow:Assets written to: /var/folders/43/4pdy4jr56rx76bghv9vbsx140000gn/T/tmpvq5_xed6/assets


INFO:tensorflow:Assets written to: /var/folders/43/4pdy4jr56rx76bghv9vbsx140000gn/T/tmpvq5_xed6/assets


RUN tensorflowjs_converter --input_format keras --output_format tfjs_layers_model built/model.h5 built/converted.tfjs
RUN OK
Validating partial models...
testing Conv2D: [50,3,1] => [47,1,16]...
also with no activation...
testing MaxPooling2D: [47,1,16] => [23,1,16]...
testing Conv2D: [23,1,16] => [22,1,16]...
also with no activation...
testing Conv2D: [22,1,16] => [21,1,16]...
also with no activation...
testing Dense: [336] => [4]...
also with no activation...
Compiling full model...
write built/model-float32.asm (91742 bytes)
write built/model-float32.js (18056 bytes)
write built/model-float32.ml4f (13052 bytes)
write built/model-float32.txt (235 bytes)


*** built/model-float32.ml4f

Accuracy: 0.9600
  231    1    0    1
    3   91    1    0
    5    0   78    0
    7    1    1   80

model: 12.75k; code: 2.46k (19.3%); arena: 4.38k; test 0.00k
total cycles: 225149 (2.680ms at 84MHz)

Validating partial models...
testing Conv2D: [50,3,1] => [47,1,16]...
also with no activation...
tes

## See model sizes

In [27]:
! ls -l built/*.tflite built/*.ml4f

-rw-r--r-- 1 michal staff  8112 Oct 16 15:20 built/model-float16.ml4f
-rw-r--r-- 1 michal staff 13052 Oct 16 15:20 built/model-float32.ml4f
-rw-r--r-- 1 michal staff 13420 Oct 16 15:20 built/model-float32.tflite
-rw-r--r-- 1 michal staff  8480 Oct 16 15:20 built/model-int8.tflite


## Test the Models

In [28]:
def show_eval(fn, pred):
    ok = ys_test.argmax(axis=1)
    numok = 0
    for i in range(len(ok)):
        if ok[i] == pred[i]:
            numok += 1
    print("\n*** %s\n" % fn)
    print("Accuracy: %f" % (numok / len(ok)))
    print(tf.math.confusion_matrix(ok, pred))

show_eval("built/model.h5", model.predict(xs_test).argmax(axis=1))

#loss = model.evaluate(xs_validate, ys_validate)
#predictions = model.predict(xs_validate)
#print(tf.math.confusion_matrix(ys_validate.argmax(axis=1), predictions.argmax(axis=1)))

def eval_tflite(fn):
    model_q = tf.lite.Interpreter(fn)
    model_q.allocate_tensors()
    inp_ten = model_q.get_input_details()[0]["index"]
    model_input = model_q.tensor(inp_ten)
    model_output = model_q.tensor(model_q.get_output_details()[0]["index"])
    num = len(xs_test)
    model_predictions = np.empty(num)
    outputs = []

    # Run each model's interpreter for each value and store the results in arrays
    for i in range(num):
        #print(xs_test[i].flatten())
        #print(model_input)
        model_q.set_tensor(inp_ten, xs_test[i].reshape(1,50,3,1))
        #model_input().set(xs_test[i].flatten())
        model_q.invoke()
        outputs.append(model_output()[0].copy())
    
    outputs = np.array(outputs).argmax(axis=1)
    show_eval(fn, outputs)

eval_tflite("built/model-float32.tflite")
eval_tflite("built/model-int8.tflite")

# show eval of ml4f models
! cat built/model-*.txt


*** built/model.h5

Accuracy: 0.960000
tf.Tensor(
[[231   3   5   7]
 [  1  91   0   1]
 [  0   1  78   1]
 [  1   0   0  80]], shape=(4, 4), dtype=int32)

*** built/model-float32.tflite

Accuracy: 0.960000
tf.Tensor(
[[231   3   5   7]
 [  1  91   0   1]
 [  0   1  78   1]
 [  1   0   0  80]], shape=(4, 4), dtype=int32)

*** built/model-int8.tflite

Accuracy: 0.966000
tf.Tensor(
[[233   5   3   5]
 [  1  92   0   0]
 [  0   1  79   0]
 [  1   1   0  79]], shape=(4, 4), dtype=int32)

*** built/model-float16.ml4f

Accuracy: 0.9600
  231    1    0    1
    3   91    1    0
    5    0   78    0
    7    1    1   80

model: 7.92k; code: 2.51k (31.7%); arena: 4.38k; test 0.00k
total cycles: 227036 (2.703ms at 84MHz)


*** built/model-float32.ml4f

Accuracy: 0.9600
  231    1    0    1
    3   91    1    0
    5    0   78    0
    7    1    1   80

model: 12.75k; code: 2.46k (19.3%); arena: 4.38k; test 0.00k
total cycles: 225149 (2.680ms at 84MHz)



## That's it!